In [1]:
"""Calculate the effectiveness of various options strategies"""

# CORE simulation parameters:
MEAN = 0.077  # expected yearly log returns (8%?)
MARKET_VOLATILITY = .19 # market annual volatility
STOCK_VOLATILITY = 0.58 # stock of interest standard deviation
STOCK_CORREL = 0.5 # correlation of stock movement to market

beta = STOCK_CORREL * STOCK_VOLATILITY / MARKET_VOLATILITY
  

# First we generate correlated GBM representing stock and broader-market
import numpy as np
import gbm
import alpha_beta
import finance_helpers as fh
import matplotlib.pyplot as plt
from monte_carlo import monte_carlo
 
# Two data points are interesting -- 2 month out and 4.75 months out
sim_times = [2/12.0, 4.75/12.0]

mean = np.ones(shape=(1,2)) * MEAN
mean[0][1] = beta * (MEAN - fh.RISK_FREE_RATE) + fh.RISK_FREE_RATE  # Capital asset pricing model
stdev = np.ones(shape=(1,2)) * MARKET_VOLATILITY
stdev[0][1] = STOCK_VOLATILITY
corr_matrix = gbm.correl_matrix_from_corr(STOCK_CORREL)

# run simulation
sim_data = []
for sim_time in sim_times:
    S = monte_carlo(gbm.multi_generate_brownian_motion_single_step,
                    tries=1500000, # huge amount of tries needed.. 
                    total_time=sim_time,
                    mean=mean,
                    stdev=stdev,
                    corr_matrix=corr_matrix
                    )
    sim_data.append(S)

In [6]:
# In this cell we do the math

CONCENTRATION = 0.50 # portfolio concentration

COLLAR_MIN = 0.928

def print_stats(return_set, risk_free_rate):
    print 'return %0.2f%%' % (100*fh.average_log_return(return_set))
    print 'volatility %0.2f%%' % (100*fh.volatility_of_returns(return_set))
    print 'Sharpe %0.2f%%'% (100*fh.sharpe_ratio(return_set, risk_free_rate))
    

def analyze(return_set, collar_max, premium, risk_free_rate=fh.RISK_FREE_RATE,):
    """analyze the returns"""
    #print return_set
    spy_returns, stock_returns = zip(*return_set)
    print 'SPY'
    print_stats(spy_returns, risk_free_rate)
    
    print '\nstock'
    print_stats(stock_returns, risk_free_rate)
    
    blended = fh.blend_returns(stock_returns, spy_returns, CONCENTRATION)
    print '\nunhedged blend concentration %2.f%%:' % (100*CONCENTRATION)
    print_stats(blended, risk_free_rate)
    
    
    collared_returns = fh.apply_collar(stock_returns, min_strike=COLLAR_MIN,
                                    max_strike=collar_max)
    blended = fh.blend_returns(collared_returns, spy_returns, CONCENTRATION)
    print '\ncollared:'
    print_stats(blended, risk_free_rate)
    
    # premium multiplier: We assume the premium from call is invested in SPY and from put, removed\n",
    premium_multiplier = premium*CONCENTRATION/(1-CONCENTRATION)
    
    #print 'pm', premium_multiplier
    # pure puts    
    put_returns = fh.apply_protected_put(stock_returns,
                                strike=COLLAR_MIN)
    spy_returns_op_cost = [(ret * (1-premium_multiplier)) for ret in spy_returns]
    blended = fh.blend_returns(put_returns, spy_returns_op_cost, CONCENTRATION)
    print '\nputs:'
    print_stats(blended, risk_free_rate)
    
    # pure calls
    call_returns = fh.apply_covered_call(stock_returns, 
                            strike=collar_max)
    spy_returns_excess = [ret * (1+premium_multiplier) for ret in spy_returns]
    blended = fh.blend_returns(call_returns, spy_returns_excess, CONCENTRATION)
    print '\ncalls:'
    print_stats(blended, risk_free_rate)
    
    
# zero priced collar w.r.t. collar_min
collar_max =  {
    2/12.0 : 1.097, 
    4.75/12.0 : 1.119
}
premium = {
    2/12.0 : 0.062, # at ~53% IV
    4.75/12.0 : 0.089 # ~53% IV
}

for sim_time, return_set in zip(sim_times, sim_data):
    print '\n%0.2f months out' % (sim_time * 12)
    analyze(return_set, collar_max=collar_max[sim_time],
            premium=premium[sim_time],
            risk_free_rate=fh.RISK_FREE_RATE*sim_time)


2.00 months out
SPY


return 1.27%


volatility 7.76%


Sharpe 13.28%



stock
return 1.87%

...


Sharpe 19.35%



calls:


return 2.24%


volatility 15.13%


Sharpe 11.02%